In [9]:
import pandas as pd
import numpy as np


In [11]:
df = pd.read_csv('data/events_up_to_01062018.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [12]:
df.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
test = df.groupby('person')['timestamp'].value_counts().to_frame()

In [14]:
test.columns = ['contar']

In [15]:
test.reset_index(inplace=True)

In [16]:
test

,person,timestamp,contar
0,0008ed71,2018-05-17 12:27:47,1
1,0008ed71,2018-05-17 13:44:59,1
2,0008ed71,2018-05-17 13:45:00,1
3,0008ed71,2018-05-17 16:21:54,1
4,0008ed71,2018-05-17 16:22:06,1
5,0008ed71,2018-05-17 16:28:37,1
6,00091926,2018-05-08 23:30:44,3
7,00091926,2018-05-09 00:25:05,3
8,00091926,2018-05-11 02:23:38,3
9,00091926,2018-05-26 03:02:01,3


In [17]:
test.sort_values(['person', 'timestamp'], ascending=True,inplace=True)

In [ ]:
test

,person,timestamp,contar
0,0008ed71,2018-05-17 12:27:47,1
1,0008ed71,2018-05-17 13:44:59,1
2,0008ed71,2018-05-17 13:45:00,1
3,0008ed71,2018-05-17 16:21:54,1
4,0008ed71,2018-05-17 16:22:06,1
5,0008ed71,2018-05-17 16:28:37,1
34,00091926,2018-05-03 22:08:29,1
35,00091926,2018-05-03 22:08:35,1
36,00091926,2018-05-03 22:08:51,1
37,00091926,2018-05-03 22:09:25,1


In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

persona_antes=None
test["conte_10"] = False

f = IntProgress(min=0, max=2023613) # instantiate the bar
display(f) # display the bar


usuario_ya_con_conte = False   
for i, fila in test.iterrows():
    f.value += 1 # signal to increment the progress bar
    contar=0
    if persona_antes is None or persona_antes != fila["person"]:
        usuario_ya_con_conte=False
        persona_antes=fila["person"]
        df_aux = test[test["person"]==fila["person"]]
    if usuario_ya_con_conte: continue
        
    for i2, fila2 in df_aux.iterrows():
        if (abs(fila['timestamp'] - fila2['timestamp'])).seconds < 60*20: # 20 minutos
            contar+=test.at[i,"contar"]
        else:
            break
            
        if contar==10:
            test.at[i,"conte_10"] = True
            usuario_ya_con_conte = True
            break
    

A Jupyter Widget

In [ ]:
test.to_csv("data/salvo.csv", sep=',')

In [ ]:
test[test["conte_10"]==True].to_csv("data/conte_10.csv",sep=",")

In [ ]:
test

In [ ]:
usuarios_solo_con_true = test[test["conte_10"]==True]

In [ ]:
personas_sin_repetir = df.drop_duplicates(subset="person", keep='first')

In [ ]:
personas_sin_repetir = personas_sin_repetir.loc[:,"person"]

In [ ]:
personas_sin_repetir = personas_sin_repetir.to_frame()

In [ ]:
type(personas_sin_repetir.at[0,"person"])

In [ ]:
usuarios_solo_con_true

In [ ]:
personas_sin_repetir.set_index("person",inplace=True)
usuarios_solo_con_true.set_index("person",inplace=True)

In [ ]:
df_conte = personas_sin_repetir.join(usuarios_solo_con_true,on="person",how="left")

In [ ]:
df_final = df_conte.loc[:,"conte_10"].to_frame().fillna(False)

In [ ]:
df_final.to_csv("data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv", sep=',')